In [234]:
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import os
from glob import glob
from tqdm import tqdm
import imageio
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import f1_score, jaccard_score, precision_score, recall_score, accuracy_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, Flatten
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.metrics import Recall, Precision, IoU
from albumentations import HorizontalFlip, VerticalFlip, ElasticTransform, GridDistortion, OpticalDistortion, CoarseDropout

In [112]:
H = 512
W = 512

In [46]:
data_path = r"E:\python\segmentation\Computer Vision\Data\blood/"

In [48]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [50]:
def load_data(path):
    """X == images , Y == masks"""
    train_x = sorted(glob(os.path.join(path, "training", "images", "*.tif")))
    train_y = sorted(glob(os.path.join(path, "training", "1st_manual", "*.gif")))

    test_x = sorted(glob(os.path.join(path, "test", "images", "*.tif")))
    test_y = sorted(glob(os.path.join(path, "test", "1st_manual", "*.gif")))

    return (train_x, train_y), (test_x, test_y)

In [52]:
def augmentation_data(images, masks, save_path, augment=True):
    H = 512
    W = 512

    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        #print(x, y)
        name = x.split("\\")[-1].split("\\")[0]
        print(name)

        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = imageio.mimread(y)[0]
        print(x.shape, y.shape)

        if augment == True:
            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            X1 = augmented["image"]
            y1 = augmented["mask"]

            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            X2 = augmented["image"]
            y2 = augmented["mask"]

            aug = ElasticTransform(p=1.0)
            augmented = aug(image=x, mask=y)
            X3 = augmented["image"]
            y3 = augmented["mask"]

            aug = GridDistortion(p=1.0)
            augmented = aug(image=x, mask=y)
            X4 = augmented["image"]
            y4 = augmented["mask"]

            aug = OpticalDistortion(p=1.0)
            augmented = aug(image=x, mask=y)
            X5 = augmented["image"]
            y5 = augmented["mask"]

            aug = CoarseDropout(p=1.0)
            augmented = aug(image=x, mask=y)
            X6 = augmented["image"]
            y6 = augmented["mask"]

            X = [x, X1, X2, X3, X4, X5, X6]
            y = [y, y1, y2, y3, y4, y5, y6]

        else:
            X = [x]
            y = [y]

        index = 0
        for i, m in zip(X, y):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))

            if len(X) == 1:
                tmp_image_name = f"{name}.jpg"
                tmp_mask_name = F"{name}.jpg"

            else:
                tmp_image_name = f"{name}_{index}.jpg"
                tmp_mask_name = F"{name}_{index}.jpg"

            image_path = os.path.join(save_path, "image", tmp_image_name)
            mask_path = os.path.join(save_path, "mask", tmp_mask_name)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            index += 1
            
        

In [179]:
if __name__ == "__main__":
    """Seeding"""
    np.random.seed(42)

    """Load the data"""
    data_path = r"E:\python\segmentation\Computer Vision\Data\blood/"
    (X_train, y_train), (X_test, y_test) = load_data(data_path)


    """Creating directories"""
    create_dir(data_path + "new_data/train/image")
    create_dir(data_path + "new_data/train/mask")    
    create_dir(data_path + "new_data/test/image")
    create_dir(data_path + "new_data/test/mask")
    
    save_path_train = r"E:\python\segmentation\Computer Vision\Data\blood\new_data\train"
    save_path_test = r"E:\python\segmentation\Computer Vision\Data\blood\new_data\test"
    augmentation_data(X_train, y_train, save_path_train , augment=True)
    augmentation_data(X_test, y_test, save_path_test , augment=False)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

21_training.tif
(584, 565, 3) (584, 565)


  5%|████▏                                                                              | 1/20 [00:00<00:10,  1.84it/s]

22_training.tif
(584, 565, 3) (584, 565)


 10%|████████▎                                                                          | 2/20 [00:01<00:09,  1.85it/s]

...

40_training.tif
(584, 565, 3) (584, 565)


 35%|█████████████████████████████                                                      | 7/20 [00:00<00:00, 60.23it/s]

01_test.tif
(584, 565, 3) (584, 565)
02_test.tif
(584, 565, 3) (584, 565)
03_test.tif
(584, 565, 3) (584, 565)
04_test.tif
(584, 565, 3) (584, 565)
05_test.tif
(584, 565, 3) (584, 565)
06_test.tif
(584, 565, 3) (584, 565)
07_test.tif
(584, 565, 3) (584, 565)
08_test.tif
(584, 565, 3) (584, 565)
09_test.tif
(584, 565, 3) (584, 565)
10_test.tif
(584, 565, 3) (584, 565)
11_test.tif
(584, 565, 3) (584, 565)
12_test.tif
(584, 565, 3) (584, 565)


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 60.90it/s]

13_test.tif
(584, 565, 3) (584, 565)
14_test.tif
(584, 565, 3) (584, 565)
15_test.tif
(584, 565, 3) (584, 565)
16_test.tif
(584, 565, 3) (584, 565)
17_test.tif
(584, 565, 3) (584, 565)
18_test.tif
(584, 565, 3) (584, 565)
19_test.tif
(584, 565, 3) (584, 565)
20_test.tif
(584, 565, 3) (584, 565)


In [54]:
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

In [56]:
def encoder_block(input, num_filters):
    x= conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

In [58]:
def decoder_block(input, skip_fetures, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_fetures])
    x = conv_block(x, num_filters)
    return x

In [60]:
def build_unet(inpput_shape):
    inputs = Input(inpput_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="Unet")
    
    return model

In [62]:
if __name__ == "__main__":
    input_shape = (512, 512, 3)
    model = build_unet(input_shape)
    model.summary()

Model: "Unet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv2d_19 (Conv2D)          (None, 512, 512, 64)         1792      ['input_2[0][0]']             
                                                                                                  
 batch_normalization_18 (Ba  (None, 512, 512, 64)         256       ['conv2d_19[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_18 (Activation)  (None, 512, 512, 64)         0         ['batch_normalization_18[0]

In [15]:
def load_dataset(path):
    x = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y = sorted(glob(os.path.join(path, "mask", "*.jpg")))
    return x, y

In [17]:
def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

In [34]:
def read_images(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    #x = cv2.resize(x , (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x

In [36]:
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    #x = cv2.resize(x , (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

In [38]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_images(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

In [42]:
def tf_dataset(X, y, batch=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(4)
    return dataset

In [64]:
smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = Flatten()(y_true)
    y_pred = Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2 * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

In [66]:
def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [100]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intesection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intesection
        x = (intesection + smooth) / (union + smooth)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

In [106]:
dataset_path = r"E:\python\segmentation\Computer Vision\Data\blood\new_data"
if __name__ == "__main__":
    np.random.seed(42)
    tf.random.set_seed(42)

    create_dir(dataset_path + "/files")

    
    batch_size = 2
    learning_rate = 1e-4
    num_epochs = 20
    model_path = os.path.join(dataset_path + "files" + "Retina_Blood_Verssel.h5")
    csv_path = os.path.join(dataset_path + "files" + "retina_blood_verssel.csv")

    train_path = os.path.join(dataset_path, "train/")
    
    validation_path = os.path.join(dataset_path, "test/")
    

    X_train, y_train = load_dataset(train_path)
    X_train, y_train = shuffling(X_train, y_train)

    X_val, y_val = load_dataset(validation_path)
    print(F"Training dataset : {len(X_train)} - {len(y_train)}")
    print(F"Validation dataset : {len(X_val)} - {len(y_val)}")

    train_dataset = tf_dataset(X_train, y_train, batch=batch_size)
    val_dataset = tf_dataset(X_val, y_val, batch=batch_size)

    train_steps = len(X_train) // batch_size
    val_steps = len(X_val) // batch_size

    if len(X_train) % batch_size != 0:
        train_steps += 1

    if len(X_val) % batch_size != 0:
        val_steps += 1

    model = build_unet((H, W, 3))
    model.compile(loss=dice_loss, optimizer=Adam(learning_rate=learning_rate), 
                 metrics=[dice_coef, iou, Recall(), Precision()])

    model.summary()


Training dataset : 140 - 140
Validation dataset : 20 - 20
Model: "Unet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv2d_152 (Conv2D)         (None, 512, 512, 64)         1792      ['input_9[0][0]']             
                                                                                                  
 batch_normalization_144 (B  (None, 512, 512, 64)         256       ['conv2d_152[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_144 (Activation  (None, 5

In [78]:
callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr1=1e-6, verbose=1),
        CSVLogger(csv_path),
        #TensorBoard(),
        EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
    ]

In [108]:
model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=val_dataset,
    steps_per_epoch=train_steps,
    validation_steps=val_steps,
    callbacks=callbacks
)

Epoch 1/20
70/70 [==============================] - ETA: 0s - loss: 0.6906 - dice_coef: 0.3094 - iou: 0.1851 - recall_5: 0.7139 - precision_5: 0.3998
Epoch 1: val_loss improved from inf to 0.86270, saving model to E:\python\segmentation\Computer Vision\Data\blood\new_datafilesRetina_Blood_Verssel.h5


C:\Users\hamid\anaconda3\envs\MachineLearning\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


70/70 [==============================] - 715s 10s/step - loss: 0.6906 - dice_coef: 0.3094 - iou: 0.1851 - recall_5: 0.7139 - precision_5: 0.3998 - val_loss: 0.8627 - val_dice_coef: 0.1373 - val_iou: 0.0737 - val_recall_5: 0.0000e+00 - val_precision_5: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/20
70/70 [==============================] - ETA: 0s - loss: 0.5551 - dice_coef: 0.4449 - iou: 0.2866 - recall_5: 0.5119 - precision_5: 0.7565
Epoch 2: val_loss did not improve from 0.86270
70/70 [==============================] - 672s 10s/step - loss: 0.5551 - dice_coef: 0.4449 - iou: 0.2866 - recall_5: 0.5119 - precision_5: 0.7565 - val_loss: 0.8838 - val_dice_coef: 0.1162 - val_iou: 0.0617 - val_recall_5: 0.0000e+00 - val_precision_5: 0.0000e+00 - lr: 1.0000e-04
Epoch 3/20
70/70 [==============================] - ETA: 0s - loss: 0.5105 - dice_coef: 0.4895 - iou: 0.3245 - recall_5: 0.4958 - precision_5: 0.8215
Epoch 3: val_loss did not improve from 0.86270
70/70 [==============================] - 675s 

In [116]:
model.save("Retina_Blood_Vessel.h5")

In [166]:
def read_test_images(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    ori_x = x
    x = x / 255.0
    x = x.astype(np.float32)
    return ori_x, x

In [228]:
def read_test_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    ori_x = x
    x = x / 255.0
    x = x.astype(np.int32)
    return ori_x, x

In [180]:
def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255
    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    concatinate_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)

    cv2.imwrite(save_image_path, concatinate_images)
    

In [238]:
dataset_path = r"E:\python\segmentation\Computer Vision\Data\blood\new_data"

retina_blood_model_path = r"E:\python\segmentation\Computer Vision\Data\blood\new_datafilesRetina_Blood_Verssel.h5"
if __name__ == "__main__":
    create_dir(dataset_path + "results")

    with CustomObjectScope({"iou":iou, "dice_coef":dice_coef, "dice_loss":dice_loss}):
        retina_blood_model = load_model(retina_blood_model_path)
        #retina_blood_model.summary()

    test_dataset_path = os.path.join(dataset_path, "test")
    X_test, y_test = load_dataset(test_dataset_path)
    #print(X_test, y_test)

    score = []
    for x, y in tqdm(zip(X_test, y_test), total=len(X_test)):
        #print(x, y)
        name = x.split("\\")[-1].split(".")[0]
        print(name)

        ori_x, x = read_test_images(x)
        ori_y, y = read_test_mask(y)

        y_pred = retina_blood_model.predict(np.expand_dims(x, axis=0))[0]
        y_pred = y_pred > 0.5
        #print(y_pred.shape)
        y_pred = y_pred.astype(np.int32)
        y_pred = np.squeeze(y_pred, axis=-1)
        #print(y_pred.shape)

        save_path = r"E:\python\segmentation\Computer Vision\Data\blood\new_data\results"
        save_image_path = f"result{name+'A'}.png"
        save_image_path = os.path.join(save_path, save_image_path)
        save_results(ori_x, ori_y, y_pred, save_image_path) 

        y = y.flatten()
        y_pred = y_pred.flatten()

        accuracy_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")

        score.append([name, accuracy_value, f1_value, jac_value, recall_value, precision_value])

    score = [s[1:] for s in score]
    score = np.mean(score, axis=0)

    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")
        

    df = pd.DataFrame(score, columns=["Image", "Accuracy", "F1_score", "Jaccard", "Recall", "Precision"])
    df.to_csv(save_path + "\\score.csv")

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

01_test
1/1 [==============================] - 4s 4s/step


  5%|████▏                                                                              | 1/20 [00:04<01:23,  4.41s/it]

02_test
1/1 [==============================] - 1s 910ms/step


 10%|████████▎                                                                          | 2/20 [00:05<00:46,  2.59s/it]

...

19_test
1/1 [==============================] - 1s 891ms/step


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [00:27<00:01,  1.30s/it]

20_test
1/1 [==============================] - 1s 943ms/step


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:29<00:00,  1.46s/it]

Accuracy: 0.94709
F1: 0.52832
Jaccard: 0.35963
Recall: 0.88130
Precision: 0.38008
